In [37]:
%pip install pandas


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: C:\Users\chahi\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [ ]:
%pip install datasets

In [36]:
%pip install torch

     ---------------------------------------- 0.0/159.8 MB ? eta -:--:--
     ---------------------------------------- 0.2/159.8 MB 4.1 MB/s eta 0:00:39
     ---------------------------------------- 0.4/159.8 MB 4.0 MB/s eta 0:00:40
     ---------------------------------------- 0.6/159.8 MB 4.1 MB/s eta 0:00:40
     ---------------------------------------- 0.8/159.8 MB 4.1 MB/s eta 0:00:39
     ---------------------------------------- 1.0/159.8 MB 4.1 MB/s eta 0:00:40
     ---------------------------------------- 1.2/159.8 MB 4.1 MB/s eta 0:00:39
     ---------------------------------------- 1.4/159.8 MB 4.1 MB/s eta 0:00:39
     ---------------------------------------- 1.5/159.8 MB 4.1 MB/s eta 0:00:39
     ---------------------------------------- 1.7/159.8 MB 4.0 MB/s eta 0:00:40
     ---------------------------------------- 1.9/159.8 MB 4.0 MB/s eta 0:00:40
      --------------------------------------- 2.1/159.8 MB 4.0 MB/s eta 0:00:40
      --------------------------------------- 2


[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: C:\Users\chahi\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [39]:
import pandas as pd
import numpy as np
from datasets import load_dataset
import json
import ast
import torch
from torch.utils.data import DataLoader, TensorDataset, random_split
from transformers import GPT2Tokenizer, GPT2ForSequenceClassification, AdamW
from tqdm import tqdm

OSError: [WinError 126] The specified module could not be found. Error loading "C:\Users\chahi\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\torch\lib\shm.dll" or one of its dependencies.

In [10]:
dataset = load_dataset("daily_dialog")

In [11]:
train = pd.DataFrame(dataset["train"])
validation = pd.DataFrame(dataset["validation"])
test = pd.DataFrame(dataset["test"]) 

In [21]:
# train
# validation
# test

In [22]:
def format_GPT2_last_utterance(row):
    # Convert string representations into actual Python objects/lists
    dialog = row['dialog']
    emotions = row['emotion']

    formatted_dialog_GPT2 = ""
    for i, (utterance, emotion) in enumerate(zip(dialog, emotions)):

        speaker = "<Speaker_1>" if i % 2 == 0 else "<Speaker_2>"
        if i < len(dialog)-1:
            formatted_dialog_GPT2 += f"{speaker}: {utterance} | <Emotion>: {emotion} <|endofutterance|> "
        else:
            formatted_dialog_GPT2 += f"{speaker}: {utterance} | <Emotion>: "

    last_utterance_emotion = emotions[-1]

    return formatted_dialog_GPT2, last_utterance_emotion

In [23]:
def format_GPT2_no_last_utterance(row):
    dialog = ast.literal_eval(row['dialog'])[:-1]  # Convert string to list and remove last utterance
    labels = row['classification']

    # Map classification to numeric labels
    label_map = {'No Derailment': 0, 'Positive Derailment': 1, 'Negative Derailment': 2}
    numeric_label = label_map[labels]

    formatted_dialog_GPT2 = ""
    for i, utterance in enumerate(dialog):
        speaker = "<Speaker_1>" if i % 2 == 0 else "<Speaker_2>"
        formatted_dialog_GPT2 += f"{speaker}: {utterance}<|endofutterance|> "

    return pd.Series([formatted_dialog_GPT2, numeric_label])

## Formatting the training set

In [25]:
formatted_dialogs = []
labels = []
for _, row in train.iterrows():
    formatted_dialog, label = format_GPT2_last_utterance(row)
    formatted_dialogs.append(formatted_dialog)
    labels.append(label)


In [ ]:
# formatted_dialogs

## Formatting the Validation set

In [30]:
formatted_dialogs_val = []
labels_val = []
for _, row in test.iterrows():
    formatted_dialog_val, label_val = format_GPT2_last_utterance(row)
    formatted_dialogs_val.append(formatted_dialog_val)
    labels_val.append(label_val)

In [32]:
# formatted_dialogs_val

## Formatting the test set

In [28]:
formatted_dialogs_test = []
labels_test = []
for _, row in test.iterrows():
    formatted_dialog_test, label_test = format_GPT2_last_utterance(row)
    formatted_dialogs_test.append(formatted_dialog_test)
    labels_test.append(label_test)

In [ ]:
# formatted_dialogs_test

In [33]:
# SETTING UP T4 GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

ModuleNotFoundError: No module named 'torch'